## Pip Installs

In [ ]:
!pip install flask -q
!pip install flask_cors -q
!pip install flask-ngrok -q
!pip install yt_dlp -q
!pip install pyngrok -q
!pip install openai -q
!pip install cohere -q
!pip install tiktoken -q
!pip install git+https://github.com/openai/whisper.git -q
!pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## Static Values

In [ ]:
!ngrok authtoken 2XE8FAke1EOuVRQhhzUB1yRfDk3_KesZbXZevP2YDxrgYaA1

OUTPUT_VIDEO = "/content/Drive/MyDrive/DownloadedVideos/downloadedVideo.mkv"
OUTPUT_AUDIO = "/content/Drive/MyDrive/OutcomeAudios/extractedAudio.wav"
OUTPUT_AUDIO_CONVERTED = "/content/Drive/MyDrive/OutcomeAudios/finalAudio.wav"
AUDIO_FOLDER = "/content/Drive/MyDrive/OutcomeAudios/"
VIDEO_FOLDER = "/content/Drive/MyDrive/DownloadedVideos/"
OUTPUT_TEXT_FILE = "script.txt"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import os

def checkAndMakeDir(folder):
  if os.path.exists(folder)==False:
    os.makedirs(folder)

In [ ]:
checkAndMakeDir(AUDIO_FOLDER)
checkAndMakeDir(VIDEO_FOLDER)
checkAndMakeDir("static")
checkAndMakeDir("templates")

## Step 1 - Video To Text

In [ ]:
from yt_dlp import YoutubeDL

def downloadVideoYT(URL):
  ytdl_format_options = {
    'outtmpl': OUTPUT_VIDEO,
    'format': 'best[ext=mp4]'
  }
  with YoutubeDL(ytdl_format_options) as ydl:
    ydl.download([URL])
  print("Download Complete")

def extractAudio():
  import subprocess
  subprocess.call(["ffmpeg",
                   "-y",
                   "-i",
                   OUTPUT_VIDEO,
                   OUTPUT_AUDIO])
  print("Audio Extraction Complete")

In [ ]:
import whisper

def initializeWhisper():
  transcriptModel = whisper.load_model('base')
  return transcriptModel

model = initializeWhisper()

def useOpenAIWhisper():
  result = model.transcribe(OUTPUT_AUDIO)
  return result['text']

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 135MiB/s]


## Step 2 - Machine Translation

In [ ]:
!pip install mtranslate -q

  Preparing metadata (setup.py) ... done


In [ ]:
# Import the 'mtranslate' library
from mtranslate import translate

def translateText(text, toLanguage, fromLanguage):
    try:
        translated_text = translate(text, toLanguage, fromLanguage)
        return translated_text
    except Exception as e:
        return f"Translation error: {str(e)}"

## Step 3 - Text to Audio

In [ ]:
#!pip install git+https://github.com/suno-ai/bark.git
#!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
#from bark import SAMPLE_RATE, generate_audio, preload_models
#from scipy.io.wavfile import write as write_wav
#from IPython.display import Audio
#import scipy
#import os
#import nltk  # we'll use this to split into sentences
#import numpy as np
#import soundfile as sf
#from bark.api import semantic_to_waveform
#from bark import generate_audio, SAMPLE_RATE


#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# download and load all models
#preload_models()

#nltk.download('punkt')
#from bark.generation import (
#    generate_text_semantic,
#    preload_models,
#)

In [ ]:
def read_and_clean_script(script_file):
    try:
        with open(script_file, "r") as file:
            script = file.read()
        script = script.replace("\n", " ").strip()
        return script
    except FileNotFoundError:
        print(f"File '{script_file}' not found.")
        return None

def convert(speech):
  sentences = nltk.sent_tokenize(script)
  SPEAKER = "v2/hi_speaker_6"
  GEN_TEMP = 0.6
  silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence
  pieces = []
  for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence, #Tokenized sentence
        history_prompt=SPEAKER, #speaker model
        temp=GEN_TEMP, #Threshold for randomness in generated voice
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )
    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

  final_piece=np.concatenate(pieces)
  sf.write("output_audio.wav", final_piece, SAMPLE_RATE)
  # Get the full path to the saved file
  file_path_audioOutput = os.path.abspath("output_audio.wav")
  return file_path_audioOutput

## Step 4 - Audio + Video

In [ ]:
!pip install moviepy

**Step 5 - Add translated audio to the video**

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

class AudioVideoUtils:
    def __init__(self, audio, video):
        self.translated_audio = AudioFileClip(audio)
        self.video = VideoFileClip(video)
        self.original_audio = self.video.audio

    def add_audio_to_video(self):
        self.trim_audio
        new_video =  self.video.set_audio(self.translated_audio)
        new_video.write_videofile("/content/static/translatedVideo.mp4")

    def trim_audio(self):
        duration_difference = self.translated_audio.duration - self.video.duration
        if duration_difference <= 0:
            return
        else:
            self.audio = self.audio.subclip(0, self.video.duration)

## Flask App

In [ ]:
# Check if the files are uploaded to the Static and Templates folder
import os
if not os.path.isfile("static/styles.css"):
  1/0
if not os.path.isfile("templates/index.html"):
  1/0

ZeroDivisionError: ignored

In [ ]:
#https://www.youtube.com/watch?v=R3AT8AtkeGc
from flask_ngrok import run_with_ngrok
from flask import Flask, redirect, url_for, render_template, request
import os
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/admin/")
def admin():
    return redirect(url_for("home"))


@app.route("/video/", methods=["GET", "POST"])
def getURL():
    if os.path.isfile(OUTPUT_VIDEO):
      os.remove(OUTPUT_VIDEO)
    if os.path.isfile(OUTPUT_AUDIO):
      os.remove(OUTPUT_AUDIO)
    if os.path.isfile('/content/static/translatedVideo.mp4'):
      os.remove("/content/static/translatedVideo.mp4")
    videoURL = request.form.get("videoUrl")
    text = getStep1Outcome(videoURL)
    translatedText = translateText(text, 'hi', 'en')
    obj = AudioVideoUtils('/content/Drive/MyDrive/output_audio.wav' , OUTPUT_VIDEO)
    obj.add_audio_to_video()
    return render_template('index.html', WhisperOutcome = text, TranslationOutcome = translatedText)

def getStep1Outcome(url):
  downloadVideoYT(url)
  extractAudio()
  out = useOpenAIWhisper()
  return out

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9a34-34-121-164-211.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:03:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:03:00] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:03:00] "GET /static/translatedVideo.mp4 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:03:00] "GET /favicon.ico HTTP/1.1" 404 -


[youtube] Extracting URL: https://www.youtube.com/watch?v=R3AT8AtkeGc
[youtube] R3AT8AtkeGc: Downloading webpage
[youtube] R3AT8AtkeGc: Downloading ios player API JSON
[youtube] R3AT8AtkeGc: Downloading android player API JSON
[youtube] R3AT8AtkeGc: Downloading m3u8 information
[info] R3AT8AtkeGc: Downloading 1 format(s): 22
[download] Destination: /content/Drive/MyDrive/DownloadedVideos/downloadedVideo.mkv
[download] 100% of   10.59MiB in 00:00:01 at 10.26MiB/s  
Download Complete
Audio Extraction Complete
Moviepy - Building video /content/static/translatedVideo.mp4.
MoviePy - Writing audio in translatedVideoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/static/translatedVideo.mp4



t: 100%|██████████| 4376/4376 [03:33<00:00, 46.63it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/Drive/MyDrive/DownloadedVideos/downloadedVideo.mkv, 2764800 bytes wanted but 0 bytes read,at frame 4375/4376, at time 175.00/175.01 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:03] "POST /video/ HTTP/1.1" 200 -


Moviepy - Done !
Moviepy - video ready /content/static/translatedVideo.mp4


INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:04] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:13] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:13] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:14] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:14] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:14] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:43] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:43] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:45] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:46] "GET /static/translatedVideo.mp4 HTTP/1.1" 206 -
